In [1]:
pip install mediapipe opencv-python numpy


In [2]:
# Import necessary libraries
import cv2
import mediapipe as mp
import numpy as np
import time
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode

# Initialize MediaPipe Face Mesh
# refine_landmarks=True gives us more detailed landmarks around the eyes and lips
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)

In [14]:
# --- Helper Functions for Google Colab Video I/O ---

def video_stream():
  """
  Starts a video stream in the Colab notebook using JavaScript to access the webcam.
  """
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var pendingResolve = null;

    async function createDom() {
      if (div !== null) {
        return;
      }
      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<div style='padding-bottom: 5px;'><b>Status: </b><span id='status'></span></div>";
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.display = 'none';
      imgElement.width = div.clientWidth - 6;
      div.appendChild(imgElement);

      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "user"}});
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = video.videoWidth;
      captureCanvas.height = video.videoHeight;
      captureCanvas.getContext('2d').drawImage(video, 0, 0);

      video.style.display = 'none';
      imgElement.style.display = 'block';
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
    }

    async function takePhoto(quality) {
      await createDom();
      captureCanvas.getContext('2d').drawImage(video, 0, 0);
      const data = captureCanvas.toDataURL('image/jpeg', quality);
      return data;
    }

    function showFrame(imgData) {
        imgElement.src = 'data:image/jpeg;base64,' + imgData;
    }

    function updateStatus(status) {
        document.getElementById('status').innerText = status;
    }
  ''')
  display(js)

def take_photo(quality=0.8):
  """Captures a single frame from the video stream as a Base64 JPEG."""
  data = eval_js('takePhoto({})'.format(quality))
  return data

def show_frame(img_data):
    """Displays a frame in the Colab output."""
    eval_js(f'showFrame("{img_data}")')

def update_status(status):
    """Updates the status text in the Colab output."""
    eval_js(f'updateStatus("{status}")')


# --- Core Logic and Image Conversion Functions ---

def base64_to_cv2(b64str):
    """Decodes a Base64 string to an OpenCV image."""
    data = b64str.split(',')[1]
    img_bytes = b64decode(data)
    img_arr = np.frombuffer(img_bytes, dtype=np.uint8)
    return cv2.imdecode(img_arr, cv2.IMREAD_COLOR)

def cv2_to_base64(img):
    """Encodes an OpenCV image to a Base64 string."""
    _, buffer = cv2.imencode('.jpg', img)
    return b64encode(buffer).decode('utf-8')

def get_eye_aspect_ratio(landmarks, eye_indices):
    """Calculates the Eye Aspect Ratio (EAR) for a single eye."""
    p1 = landmarks[eye_indices[0]]
    p2 = landmarks[eye_indices[1]]
    p3 = landmarks[eye_indices[2]]
    p4 = landmarks[eye_indices[3]]
    p5 = landmarks[eye_indices[4]]
    p6 = landmarks[eye_indices[5]]

    vertical1 = ((p2.x - p6.x)**2 + (p2.y - p6.y)**2)**0.5
    vertical2 = ((p3.x - p5.x)**2 + (p3.y - p5.y)**2)**0.5
    horizontal = ((p1.x - p4.x)**2 + (p1.y - p4.y)**2)**0.5

    if horizontal == 0:
        return 0.0

    ear = (vertical1 + vertical2) / (2.0 * horizontal)
    return ear
def draw_dotted_line(frame, pt1, pt2, color, thickness=1, gap=7):
    """Draws a dotted line on a frame by plotting circles along a path."""
    dist = ((pt1[0] - pt2[0])**2 + (pt1[1] - pt2[1])**2)**.5
    pts = []
    if dist > 0:
      for i in np.arange(0, dist, gap):
          r = i / dist
          x = int((pt1[0] * (1 - r) + pt2[0] * r) + .5)
          y = int((pt1[1] * (1 - r) + pt2[1] * r) + .5)
          p = (x, y)
          pts.append(p)

    for p in pts:
        cv2.circle(frame, p, thickness, color, -1)
print("Helper functions defined successfully.")

Helper functions defined successfully.


In [ ]:
# --- Main Application Loop (V3 with EAR Visualization) ---

# Constants
EYE_AR_THRESH = 0.23         # Threshold for eye closure. Adjust as needed.
DROWSY_TIME_SECONDS = 5      # Seconds of consecutive eye closure to trigger an alert.

# State variable for the timer
drowsy_start_time = None

print("Starting video stream... Please allow camera access in your browser.")
video_stream()
print("Video stream started. Running detection loop...")

while True:
    try:
        # Capture a frame
        frame_b64 = take_photo()
        frame = base64_to_cv2(frame_b64)

        # Get frame dimensions for converting normalized landmarks to pixel coordinates
        frame_height, frame_width, _ = frame.shape

        # Process the frame
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)

        status_text = "Awake"
        display_color = (0, 255, 0) # Green

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                landmarks = face_landmarks.landmark

                # --- Define Eye Landmark Indices ---
                # These are the 6 points for each eye used in the EAR calculation
                left_eye_indices = [33, 160, 158, 133, 153, 144]
                right_eye_indices = [362, 385, 387, 263, 373, 380]

                # --- Calculate EAR ---
                left_ear = get_eye_aspect_ratio(landmarks, left_eye_indices)
                right_ear = get_eye_aspect_ratio(landmarks, right_eye_indices)
                avg_ear = (left_ear + right_ear) / 2.0

                # --- NEW: Visualization Logic ---
                # We will draw lines on both eyes
                for eye_indices in [left_eye_indices, right_eye_indices]:
                    # Get the 6 landmark points for the current eye
                    p1 = landmarks[eye_indices[0]]
                    p2 = landmarks[eye_indices[1]]
                    p3 = landmarks[eye_indices[2]]
                    p4 = landmarks[eye_indices[3]]
                    p5 = landmarks[eye_indices[4]]
                    p6 = landmarks[eye_indices[5]]

                    # Convert normalized coordinates to pixel coordinates
                    p1_px = (int(p1.x * frame_width), int(p1.y * frame_height))
                    p2_px = (int(p2.x * frame_width), int(p2.y * frame_height))
                    p3_px = (int(p3.x * frame_width), int(p3.y * frame_height))
                    p4_px = (int(p4.x * frame_width), int(p4.y * frame_height))
                    p5_px = (int(p5.x * frame_width), int(p5.y * frame_height))
                    p6_px = (int(p6.x * frame_width), int(p6.y * frame_height))

                    # Draw the lines on the frame
                    draw_dotted_line(frame, p2_px, p6_px, (0, 255, 0), thickness=1, gap=7)
                    draw_dotted_line(frame, p3_px, p5_px, (0, 255, 0), thickness=1, gap=7)
                    draw_dotted_line(frame, p1_px, p4_px, (0, 255, 0), thickness=1, gap=7)
                    # Vertical lines in green
                    #cv2.line(frame, p2_px, p6_px, (0, 255, 0), 1)
                    #cv2.line(frame, p3_px, p5_px, (0, 255, 0), 1)
                    # Horizontal line in red
                    #cv2.line(frame, p1_px, p4_px, (0, 0, 255), 1)

                # --- Display EAR and Check Drowsiness ---
                ear_text = f"EAR: {avg_ear:.2f}"
                cv2.putText(frame, ear_text, (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

                if avg_ear < EYE_AR_THRESH:
                    if drowsy_start_time is None:
                        drowsy_start_time = time.time()
                    else:
                        elapsed_time = time.time() - drowsy_start_time
                        # --- NEW: Drowsiness Timer Visualization ---
                        bar_x, bar_y, bar_width, bar_height = 50, 120, 300, 25
                        fill_ratio = elapsed_time / DROWSY_TIME_SECONDS
                        fill_width = min(bar_width, int(fill_ratio * bar_width))

                        # Draw background of the bar
                        cv2.rectangle(frame, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (128, 128, 128), -1)
                        # Draw the filling part of the bar
                        cv2.rectangle(frame, (bar_x, bar_y), (bar_x + fill_width, bar_y + bar_height), (0, 165, 255), -1) # Orange fill
                        if elapsed_time >= DROWSY_TIME_SECONDS:
                            status_text = "DROWSY!"
                            display_color = (0, 0, 255)
                else:
                    drowsy_start_time = None
                    status_text = "Awake"
                    display_color = (0, 255, 0)
        else:
            status_text = "No Face Detected"
            display_color = (255, 165, 0)
            drowsy_start_time = None

        cv2.putText(frame, status_text, (50, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, display_color, 3)
        update_status(status_text)

        processed_frame_b64 = cv2_to_base64(frame)
        show_frame(processed_frame_b64)

    except Exception as e:
        if "NotFoundError" in str(e) or "Cannot read" in str(e):
             print("Camera stream stopped by user.")
             break
        print(f"An error occurred: {e}")
        time.sleep(1)

Starting video stream... Please allow camera access in your browser.


<IPython.core.display.Javascript object>

Video stream started. Running detection loop...
